# Build an LLM Playground

---
## Learning Objectives  
- Understand tokenization and how raw text is converted into a sequence of discrete tokens
- Inspect GPT-2 and the Transformer architecture
- Learn how to load pretrained LLMs using Hugging Face
- Explore decoding strategies to generate text from LLMs
- Compare completion models with instruction-tuned models


Let's get started!

In [ ]:
# Confirm required libraries are installed and working.
import torch, transformers, tiktoken
print("torch", torch.__version__, "| transformers", transformers.__version__)
print("✅ Environment check complete. You're good to go!")

# 1 - Tokenization

A neural network cannot process raw text directly. It needs numbers.
Tokenization is the process of converting text into numerical IDs that models can understand. In this section, you will learn how tokenization works in practice and why it is an essential step in every language model pipeline.

Tokenization methods generally fall into three main categories:
1. Word-level
2. Character-level
3. Subword-level

### 1.1 - Word-level tokenization
This method splits text by whitespace and treats each word as a single token. In the next cell, you will implement a basic word-level tokenizer by building a vocabulary that maps words to IDs and writing `encode` and `decode` functions.

In [ ]:
# Creating a tiny corpus. In practice, a corpus is generally the entire internet-scale dataset used for training.
corpus = [
    "The quick brown fox jumps over the lazy dog",
    "Tokenization converts text to numbers",
    "Large language models predict the next token"
]

# Step 1: Build vocabulary (all unique words in the corpus) and mappings
vocab = []
word2id = {}
id2word = {}

for sentence in corpus:
  words = sentence.split()
  for word in words:
    if word not in vocab:
      vocab.append(word)

for idx, word in enumerate(vocab):
  word2id[word] = idx
  id2word[idx] = word

print(f"Vocabulary size: {len(vocab)} words")
print("First 15 vocab entries:", vocab[:15])


In [ ]:
def encode(text):
    # converts text to token IDs

    words = text.split()
    token_ids = []

    for word in words:
        if word in word2id:
            token_ids.append(word2id[word])
        else:
            token_ids.append(-1)
    return token_ids


def decode(ids):
    # converts token IDs back to text

    words = []

    for idx in ids:
        if idx in id2word:                   # if ID exists in dictionary
            words.append(id2word[idx])       # append corresponding word
        else:
            words.append("<UNK>")            # unknown ID → placeholder token
    return " ".join(words)

In [ ]:
# Step 3: Test your tokenizer with random sentences.
# Try a sentence with unseen words and see what happens (and how to fix it)

test_sentence_1 = "The quick brown fox"
encoded_1 = encode(test_sentence_1)
decoded_1 = decode(encoded_1)

print(encoded_1)
print(decoded_1)


### 1.2 - Character-level tokenization

In this approach, every single character (including spaces, punctuation, and even emojis) is assigned its own ID.

In the next section, we will rebuild a tokenizer using the same corpus as before, but this time with a character-level approach.
For simplicity, assume we are only using lowercase and uppercase English letters (a-z, A-Z).

In [ ]:
import string

# Step 1: Create a vocabulary that includes all uppercase and lowercase letters.
vocab = []
char2id = {}
id2char = {}


# 1. Add two special tokens for unknown characters and padding
vocab.append("<PAD>")     # Used for padding if needed
vocab.append("<UNK>")     # Represents any non-supported character

# 2. Add lowercase a–z
for ch in string.ascii_lowercase:
    vocab.append(ch)

# 3. Add uppercase A–Z
for ch in string.ascii_uppercase:
    vocab.append(ch)

# 4. Build both mapping dictionaries
for idx, ch in enumerate(vocab):
    char2id[ch] = idx
    id2char[idx] = ch


print(f"Vocabulary size: {len(vocab)} (52 letters + 2 specials)")


In [ ]:
# Step 2: Implement encode() and decode() functions to convert between text and IDs.
def encode(text):
    # convert text to list of IDs
    encoded_ids = []
    for ch in text:
        if ch in char2id:
            encoded_ids.append(char2id[ch])
        else:
            encoded_ids.append(char2id["<UNK>"])
    return encoded_ids


def decode(ids):
    # Convert list of IDs back to text
    decoded_chars = []
    for idx in ids:
        if idx in id2char:
            decoded_chars.append(id2char[idx])
        else:
            decoded_chars.append("<UNK>")
    return "".join(decoded_chars)

In [ ]:
# Step 3: Test your tokenizer on a short sample word.
sample_text = "HelloWorld?!"       # includes an unsupported "!" to test <UNK>
encoded_output = encode(sample_text)
decoded_output = decode(encoded_output)

print("Original Text:", sample_text)
print("Encoded IDs  :", encoded_output)
print("Decoded Text :", decoded_output)

### 1.3 - Subword-level tokenization

Sub-word methods such as `Byte-Pair Encoding (BPE)`, `WordPiece`, and `SentencePiece` **learn** common groups of characters and merge them into tokens. For example, the word **unbelievable** might turn into three tokens: **["un", "believ", "able"]**. This approach strikes a balance between word-level and character-level methods and fix their limitations.

The BPE algorithm builds a vocabulary iteratively using the following process:
1. Start with individual characters (each character is a token).
2. Count all adjacent pairs of tokens in a large text corpus.
3. Merge the most frequent pair into a new token.


In [ ]:
from transformers import AutoTokenizer

# Step 1: Load a pretrained GPT-2 tokenizer from Hugging Face.
# Refer to this to learn more: https://huggingface.co/docs/transformers/en/model_doc/gpt2

tokenizer = AutoTokenizer.from_pretrained("gpt2")


In [ ]:
# Step 2: Use it to write encode and decode helper functions
def encode(text):
     return tokenizer.encode(text)


def decode(ids):
        return tokenizer.decode(text)

In [ ]:
# 3. Inspect the tokens to see how BPE breaks words apart.
sample = "Unbelievable tokenization powers! 🚀"

# 3. Inspect the tokens to see how BPE breaks words apart.
sample = "Unbelievable tokenization powers! 🚀"

tokens = tokenizer.tokenize(sample)   # view subword tokens
token_ids = tokenizer.encode(sample)  # numeric IDs

print("Original text:", sample)
print("Subword tokens:", tokens)
print("Token IDs:", token_ids)


### 1.4 - TikToken

`tiktoken` is a fast, production-ready library for tokenization used by OpenAI models.
It is designed for efficiency and consistency with how OpenAI counts tokens in GPT models.

In [ ]:
import tiktoken

# Compare GPT-2 and GPT-4 tokenizers using tiktoken.

# Step 1: Load two tokenizers
gpt2_tok = tiktoken.get_encoding("gpt2")
gpt4_tok = tiktoken.get_encoding("cl100k_base")

# Step 2: Encode the same sentence with both and observe how they differ
sentence = "The 🌟 star-programmer implemented AGI overnight and i love it."

gpt2_ids = gpt2_tok.encode(sentence)
gpt4_ids = gpt4_tok.encode(sentence)

print("Original text:", sentence)

print("\n=== GPT-2 TOKENIZER (gpt2) ===")
print("Token IDs:", gpt2_ids)
print("Number of tokens:", len(gpt2_ids))

print("\n=== GPT-4 TOKENIZER (cl100k_base) ===")
print("Token IDs:", gpt4_ids)
print("Number of tokens:", len(gpt4_ids))


# 2. What is a Language Model?

At its core, a **language model (LM)** is just a *very large* mathematical function built from many neural-network layers.  
Given a sequence of tokens `[t₁, t₂, …, tₙ]`, it learns to output a probability for the next token `tₙ₊₁`.

### 2.1 - A Single `Linear` Layer

Before jumping into Transformers, let's start with the simplest building block: a `Linear` layer.

A Linear layer computes `y = Wx + b`.

Where:  
  * `x` - input vector  
  * `W` - weight matrix (learned)  
  * `b` - bias vector (learned)

In [ ]:
import torch
import torch.nn as nn

# Define a MyLinear PyTorch module and perform y = Wx + b.

class MyLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(MyLinear, self).__init__()
        # Initialize weights and bias as learnable parameters.
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.randn(out_features))

    def forward(self, x):
        # Matrix multiplication followed by bias addition
        return torch.matmul(self.weight, x) + self.bias    # y = Wx + b


lin = MyLinear(3, 2)
x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))

In [ ]:
import torch.nn as nn, torch

# Create a linear layer using pytorch's nn.Linear
lin = nn.Linear(in_features=3, out_features=2)

x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))


### 2.2 - A `Transformer` Layer

Most LLMs are a **stack of identical Transformer blocks**. Each block fuses two main components:

| Step | What it does | Where it lives in code |
|------|--------------|------------------------|
| **Multi-Head Self-Attention** | Every token looks at every other token and decides *what matters*. | `block.attn` |
| **Feed-Forward Network (MLP)** | Re-mixes information token-by-token. | `block.mlp` |

In [ ]:
import torch
from transformers import GPT2LMHeadModel

# Step 1: load the smallest GPT-2 model (124M parameters) using the Hugging Face transformers library.
# Refer to: https://huggingface.co/docs/transformers/en/model_doc/gpt2
# Step 1: load the smallest GPT-2 model (124M parameters)
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Step 2: inspect (print) the first Transformer block
print(model.transformer.h[0])

In this section, you will run a minimal forward pass through one GPT-2 block to understand how tokens are transformed inside the model.

In [ ]:
# Step 1: Create a small dummy input with a sequence of 8 random token IDs.
batch_size = 2
seq_len = 16
dummy_ids = torch.randint(low=0, high=model.config.vocab_size, size=(batch_size, seq_len))

# Step 2: Convert token IDs into embeddings (token + positional)
token_embeds = model.transformer.wte(dummy_ids)                      # (B, T, hidden)
pos_ids = torch.arange(seq_len).unsqueeze(0)                         # (1, T)
pos_embeds = model.transformer.wpe(pos_ids)                          # (1, T, hidden)
hidden_states = token_embeds + pos_embeds                            # (B, T, hidden)

# Step 3: Pass the embeddings through a single Transformer block
block_out = model.transformer.h[0](hidden_states)[0]                 # first block output

# Step 4: Inspect the result
print("Output shape:", block_out.shape)



### 2.3 - Inside GPT-2

GPT-2 is essentially a stack of identical Transformer blocks arranged in sequence.
Each block contains attention, feed-forward, and normalization layers that process token representations step by step.

In [ ]:
# Print the name of all layers inside gpt.transformer.
# You may find this helpful: https://docs.pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.named_children

# Print the name of all layers inside gpt.transformer.
for name, layer in model.transformer.named_children():
    print(name, ":", type(layer))


As you can see, the Transformer holds various modules, arranged from a list of blocks (`h`). The following table summarizes these modules:

| Step | What it does | Why it matters |
|------|--------------|----------------|
| **Token → Embedding** | Converts IDs to vectors | Gives the model a numeric “handle” on words |
| **Positional Encoding** | Adds “where am I?” info | Order matters in language |
| **Multi-Head Self-Attention** | Each token asks “which other tokens should I look at?” | Lets the model relate words across a sentence |
| **Feed-Forward Network** | Two stacked Linear layers with a non-linearity | Mixes information and adds depth |
| **LayerNorm & Residual** | Stabilize training and help gradients flow | Keeps very deep networks trainable |


### 2.4 LLM's output

When you pass a sequence of tokens through a language model, it produces a tensor of logits with shape
`(batch_size, seq_len, vocab_size)`.
Each position in the sequence receives a vector of scores representing how likely every possible token is to appear next. By applying a softmax function on the last dimension, these logits can be converted into probabilities that sum to 1.


In [ ]:
import torch, torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# Step 1: Load GPT-2 model and its tokenizer

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

In [ ]:
# Step 2: Tokenize input text
text = "Hello my name"

text = "Hello my name"
inputs = tokenizer(text, return_tensors="pt")   # contains 'input_ids' and 'attention_mask'


In [ ]:
# Step 3: Pass the input IDs to the model

with torch.no_grad():
    outputs = model(**inputs)                   # forward pass
logits = outputs.logits                         # shape: (batch, seq_len, vocab_size)

print("Logits shape:", logits.shape)

In [ ]:
# Step 4: Predict the next token
# We take the logits from the final position, apply softmax to get probabilities,
# and then extract the top 5 most likely next tokens. You may find F.softmax and torch.topk helpful in your implementation.

last_token_logits = logits[0, -1, :]            # logits for final token position
probs = F.softmax(last_token_logits, dim=-1)    # convert to probabilities
top_probs, top_ids = torch.topk(probs, k=5)     # top 5 token IDs

print("\nTop 5 predicted next tokens:")
for prob, token_id in zip(top_probs, top_ids):
    print(f"{tokenizer.decode([token_id])!r}  ->  probability: {prob.item():.4f}")

# 3 - Text Generation (Decoding)
Once a language model has been trained to predict token probabilities, we can use it to generate text.
This process is called text generation or decoding.

At each step, the model outputs a probability distribution over possible next tokens.
A decoding algorithm then selects one token based on that distribution, appends it to the sequence, and repeats the process to build text word by word. Different decoding strategies control how the model chooses the next token and how creative or deterministic the output will be. For example:
- **Greedy** decoding: always pick the token with the highest probability. Simple and consistent, but often repetitive.
- **Top-k** or **Nucleus** (top-p) sampling: randomly sample from the top few likely tokens to add variety.
- Beam search: explores multiple candidate continuations and keeps the best overall sequence.

### 3.1 - Greedy decoding
In this section, you will use GPT-2 and Hugging Face's built-in generate method to produce text using the greedy decoding strategy.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


model_id = "gpt2"
device = "cuda" if torch.cuda.is_available() else "mps"


# Step 1. Load GPT-2 model and tokenizer.
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
model.eval()

# Step 2. Implement a text generation function using HuggingFace's generate method.
def generate(model, tokenizer, prompt, max_new_tokens=128):

    inputs = tokenizer(prompt, return_tensors="pt").to(device)   # tokenize and move to device
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,      # Greedy decoding
        )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [ ]:
tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n GPT-2 | Greedy")
    print(generate(model, tokenizer, prompt, 80))

### 3.2 - Top-k and top-p sampling
The generate function you implemented earlier can easily be extended to use different decoding strategies.

In this section, you will reimplement the same function but adapt it to support Top-k and Top-p (nucleus) sampling. These methods introduce controlled randomness, allowing the model to explore multiple plausible continuations instead of always choosing the single most likely next token.

In [ ]:
# Implement `generate` to support 3 strategies: greedy, top_k, and top_o
# You may find this link helpful: https://huggingface.co/docs/transformers/en/main_classes/text_generation

def generate(model, tokenizer, prompt, strategy="greedy", max_new_tokens=128):

    # Tokenize prompt and move to model's device
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Configure decoding based on chosen strategy
    if strategy == "greedy":
        # Always select highest-probability token
        gen_kwargs = dict(do_sample=False)

    elif strategy == "top_k":
        # Sample from top-k most likely tokens
        gen_kwargs = dict(
            do_sample=True,
            top_k=50,            # You may tune this (10-100 typical)
            top_p=1.0,           # Disabled for top-k
            temperature=1.0
        )

    elif strategy == "top_p":
        # Nucleus sampling: sample from tokens whose cumulative prob ≥ top_p
        gen_kwargs = dict(
            do_sample=True,
            top_k=0,             # Disabled for top-p (or leave unrestricted)
            top_p=0.9,           # You may tune this (0.8-0.95 typical)
            temperature=1.0
        )

    else:
        raise ValueError("strategy must be one of: 'greedy', 'top_k', or 'top_p'")

    # Generate output token IDs
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            **gen_kwargs
        )

    # Decode token IDs into string
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [ ]:
tests = ["Once upon a time", "What is 2+2?", "Suggest a party theme."]

for strategy in ["greedy", "top_k", "top_p"]:
    print(f"\n===== Strategy: {strategy.upper()} =====")
    for prompt in tests:
        print(f"\nPrompt: {prompt}")
        print(generate(model, tokenizer, prompt, strategy=strategy, max_new_tokens=60))

# 4 - Completion vs. Instruction-tuned LLMs

So far, we have used `GPT-2` to generate text from a given input prompt. However, `GPT-2` is just a completion model. It simply continues the provided text without understanding it as a task or question. It is not designed to engage in dialogue or follow instructions.

In contrast, instruction-tuned LLMs (such as `Qwen-Chat`) undergo an additional post-training stage after base pre-training. This process fine-tunes the model to behave helpfully and safely when interacting with users. Because of this extra stage, instruction-tuned models can:

- Interpret prompts as requests rather than just text to continue
- Stay in conversation mode, answering questions and following steps
- Handle refusals and safety boundaries appropriately
- Maintain a consistent helpful persona, rather than drifting into storytelling

### 4.1 - `Qwen/Qwen3-0.6B` vs. `GPT2`

In the next cell, you will feed the same prompt to two different models:

- GPT-2 (completion-only): continues the text in the same writing style
- Qwen/Qwen3-0.6B (instruction-tuned): interprets the input as an instruction and responds helpfully

Comparing the two outputs will make the difference between completion and instruction-tuned behavior clear.



In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load both GPT-2 and Qwen models using HuggingFace `.from_pretrained` method.

device = "cuda" if torch.cuda.is_available() else "cpu"

# ----- Load GPT-2 (Completion Model) -----
gpt2_name = "gpt2"
gpt2_tokenizer = AutoTokenizer.from_pretrained(gpt2_name)
gpt2_model = AutoModelForCausalLM.from_pretrained(gpt2_name).to(device)
gpt2_model.eval()

# ----- Load Qwen (Instruction-Tuned Model) -----
# using a small but instruction-trained chat model
qwen_name = "Qwen/Qwen2-0.5B-Instruct"
qwen_tokenizer = AutoTokenizer.from_pretrained(qwen_name)
qwen_model = AutoModelForCausalLM.from_pretrained(qwen_name).to(device)
qwen_model.eval()

print("Models loaded successfully:")
print(f"- Completion model     : {gpt2_name}")
print(f"- Instruction-tuned LLM: {qwen_name}")

We have now downloaded two small checkpoints: GPT-2 (124M parameters) and Qwen3-0.6B (600M parameters). If the previous cell took some time to run, that was mainly due to model download speed. The models will be cached locally, so future runs will be faster.

Next, we will generate text using our generate function with both models and the same prompt to directly compare how a completion-only model (GPT-2) behaves differently from an instruction-tuned model (Qwen).

In [ ]:
tests = [("Once upon a time", "greedy"),
         ("What is 2+2?", "top_k"),
         ("Suggest a party theme.", "top_p")]

for prompt, strategy in tests:
    print(f"\n=== Prompt: {prompt!r} | Strategy: {strategy} ===")

    # GPT-2 output
    gpt2_out = generate(gpt2_model, gpt2_tokenizer, prompt, strategy=strategy, max_new_tokens=80)
    print("\n[GPT-2 Completion Model]")
    print(gpt2_out)

    # Qwen output
    qwen_out = generate(qwen_model, qwen_tokenizer, prompt, strategy=strategy, max_new_tokens=80)
    print("\n[Qwen Instruction-Tuned Model]")
    print(qwen_out)
